# Analysing Genuine files in Train set - version 2.0 dataset

In [1]:
%%bash 
ls /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_train/ | wc -l

3014


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [2]:
# Load the prediction file
# Predictions file that has three columns: genuin, spoofed prob and log likehood ratio

pred_file='model_3sec_relu_0.5_run8/predictions/train_prediction.txt'
#new_pred_file = 'model_3sec_relu_0.5_run8/predictions/train_prediction_with_index.txt'
new_pred_file = 'predictions/train_prediction_with_index.txt'

In [3]:
# Create a new prediction file where we write index (starting from 0) as the first column
# This way things become much easier

'''
with open(pred_file) as f, open(new_pred_file,'w') as f2:
    i=0
    for line in f:
        f2.write(str(i)+' '+ line)
        #print(i)
        i+=1    
'''

# Above code is just used to append file index in the prediction file
# Careful when re-running

"\nwith open(pred_file) as f, open(new_pred_file,'w') as f2:\n    i=0\n    for line in f:\n        f2.write(str(i)+' '+ line)\n        #print(i)\n        i+=1    \n"

In [5]:
%%bash

#cat model_3sec_relu_0.5_run8/predictions/train_prediction_with_index.txt | head
cat predictions/train_prediction_with_index.txt | head

 #Check the new prediction file

0 0.999606 0.000393718 7.83948
1 0.998261 0.00173903 6.35269
2 0.999865 0.000135052 8.90972
3 0.997186 0.00281445 5.87017
4 0.999346 0.000653539 7.33245
5 0.99927 0.000729632 7.22224
6 0.993516 0.00648415 5.03189
7 0.99779 0.00220973 6.11267
8 0.999441 0.000558659 7.48941
9 0.995694 0.00430576 5.44349


In [6]:
# Separate out predictions of genuine and spoofed files
# Training set has first 1507 as genuine and remaining 1507 as spoofed

all_predictions = []
all_gens = []
all_spoofs = []

with open(new_pred_file) as f:
    all_predictions = [line.strip() for line in f]
    
all_gen_predictions = all_predictions[:1507]                 # first 1507 files are genuine
all_spf_predictions = all_predictions[1507:]                 # files from 1507 onwards are spoofed examples

In [7]:
print(len(all_gen_predictions))
print(len(all_spf_predictions))

1507
1507


In [8]:
# Collect all genuine file for which genuine class got probability > 90%
# the strongly correctly classified cases

'''
fname='index_file_list/allGenIndexList_TP.txt'
count=0
splitIndex = 1     # 1 for genuine and 2 for spoofed (the third column)
predList = all_gen_predictions   # use all_spf_predictions for spoofed case

#print(len(predList))

with open(fname,'w') as f2:
    for items in predList:
        prob = items.strip().split(' ')[splitIndex]   
        if float(prob) > 0.9:
            #print(items)
            count+=1
            f2.write(items+'\n')            
print(count)
'''
# Careful when you re-run this code. It is use to take files with 90% probability to do analysis using slime !

"\nfname='index_file_list/allGenIndexList_TP.txt'\ncount=0\nsplitIndex = 1     # 1 for genuine and 2 for spoofed (the third column)\npredList = all_gen_predictions   # use all_spf_predictions for spoofed case\n\n#print(len(predList))\n\nwith open(fname,'w') as f2:\n    for items in predList:\n        prob = items.strip().split(' ')[splitIndex]   \n        if float(prob) > 0.9:\n            #print(items)\n            count+=1\n            f2.write(items+'\n')            \nprint(count)\n"

In [10]:
%%bash
cat index_file_list/allGenIndexList_TP.txt | wc -l


1472


## Total genuine files we got with > 90% probability is = 1472 (out of 1507)

> Take the top 10 genuine confident correctly classified files - high scores

                               Gen prob     Spf Prob      Log Liklihood ratio
    
       
**** Note when accessing via list or array, index 575 should be called as 574       

## Analysing Time: The top two components from SLIME - True positive Genuine case

Note, that under time analysis, we have cut our input spectrogram into 10 different temporal components/segments, where each segment correpsonds to

> ***300 mili seconds***

In [11]:
# Show how we cut the spectrogram in timexfrequency

Image("time.png")

In [12]:
# Load the temporal explanantion file produced by slime

file = 'top_two_explanation_indices/time/gen_TP_box.txt'    #using box for spectrogram computation

***Make sure to use the correct spectrogram for two cases !! ***

In [13]:
%%bash 

cat top_two_explanation_indices/time/gen_TP_box.txt | wc -l

1472


In [14]:
def get_top1_top2_list(file):
    with open(file) as f:
        top1 = [int(line.strip().split(' ')[0]) for line in f]
    with open(file) as f:  
        top2 = [int(line.strip().split(' ')[1]) for line in f]
    return top1, top2    

In [15]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [16]:
print(len(top1))
print(len(top2))

1472
1472


In [17]:
def get_counts(datalist, key):
    count=0
    for i in range(0,len(datalist)):
        if datalist[i] == key:
            count+=1
    return count            

In [18]:
def print_component_distribution(comps, predList, order):
    print('Printing component weigting distribution for Top:', order)
    for i in comps:
        print('Component ' + str(i) + ' : ' + str(get_counts(predList, i)))    

### Using the top1 components - given by SLIME (Time segmentation) - Genuine Class

   
> Using only those files with 90% above genuine class probability. The pattern is same (total files this time we used is 685 earlier we used 766 that include genuine files with proba 60 % also

    Component 0 : 20
    Component 1 : 2
    Component 2 : 240
    Component 3 : 551
    Component 4 : 81
    Component 5 : 316
    Component 6 : 200
    Component 7 : 54
    Component 8 : 3
    Component 9 : 5

In [21]:
comps=[0,1,2,3,4,5,6,7,8,9]    # In time we have 10 segments/components

In [22]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 20
Component 1 : 2
Component 2 : 240
Component 3 : 551
Component 4 : 81
Component 5 : 316
Component 6 : 200
Component 7 : 54
Component 8 : 3
Component 9 : 5


# Pick 5 audio files for hearing

In [23]:
#random_5_ids = [506,480,757,208,481]   # Genuine True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [507,481,758,209,482]   # Genuine True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [24]:
%%bash
#ls /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000481.wav

In [25]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000482.wav audio_files_hearing/genuine_TP/

#play audio_files_hearing/genuine_TP/D_1000507.wav
#ls audio_files_hearing/genuine_TP/

## Analysing Freq: The top two components from SLIME - True positive Genuine case

Note, that under frequency analysis, we have cut our input spectrogram into 8 different frequency components/segments, where each segment correpsonds to

> ***1000 Hz frequency***

In [26]:
# Show how we cut the spectrogram in timexfrequency

Image("frequency.png")

In [27]:
# Load the spectral explanation file

file = 'top_two_explanation_indices/freq/gen_TP_box.txt'   #using signal box

In [28]:
%%bash
#cat 'top_two_explanation_indices/freq/gen_TP.txt' | head

# the top two components 7 6 dominates the explanation

In [29]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [30]:
print(len(top1))
print(len(top2))

1472
1472


## Top 1 component distribution - Frequency

In [31]:
comps = [0,1,2,3,4,5,6,7]   # in Frequency we have 8 components

In [32]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 0
Component 1 : 0
Component 2 : 0
Component 3 : 0
Component 4 : 0
Component 5 : 92
Component 6 : 6
Component 7 : 1374


> With 685 files chosen with > 90% probability, distribution of TOP1 explanation (its same as before)

    Component 0 : 2
    Component 1 : 0
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 9
    Component 6 : 0
    Component 7 : 674

In [240]:
# Get those file index having component8 in top explanation (as for frequency we get top explanation at index8)
top = 7

fname='index_file_list/allGenIndexList_TP.txt'
with open(fname) as f:
    #file_idxs = [int(line.strip().split(' ')[0]) for line in f]
    file_idxs = [line.strip() for line in f]
    
#Write the top component index to the file and further analysis
fname='top_two_explanation_indices/freq/topExplanation_list_gen_TP_box.txt'
id8_indexFiles=list() 

f=open(fname,'w')
for i in range(len(top1)):
    if top1[i]==top:    # if top explanation index is 3 (which is actually 4)
        id8_indexFiles.append(file_idxs[i])
        #print(file_idxs[i].strip().split(' ')[0])
        #print(file_idxs[i])        
        #f.write(str(file_idxs[i])+'\n')
        
f.close()

** Observations**

> We find that frequency component 8 corresponding to 7000-8000 Hz, seems to get maximally activated for making the prediction. Still, its hard to make any signifant conclusion about a class.



# Pick 5 audio files for hearing

In [241]:
#random_5_ids = [571,455,387,202,708]   # Genuine True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [572,456,388,203,709]   # Genuine True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [106]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000709.wav audio_files_hearing/genuine_TP/

#play audio_files_hearing/genuine_TP/D_1000507.wav
#ls audio_files_hearing/genuine_TP/

# On Training set

The training set has 1507 genuine and spoofed audio files. Note that we have used only the training dataset to train our end-to-end CNN system. On training set our model show an EER of about 0.26. Therefore, we analyse what has the CNN learned/exploited about these two classes of data.


## What CNN has exploited about Genuine signal from the training data ?

We first collect all the genuine audio files that has been strongly correctly classified with more than 90% probability by the CNN model. We find 1472 (out of 1507). Next, we run SLIME algorithm on these audio files and generate temporal and spectral explanations.

Distribution of **temporal explanation** for 

    Component 0 : 20
    Component 1 : 2
    Component 2 : 240
    Component 3 : 551
    Component 4 : 81
    Component 5 : 316
    Component 6 : 200
    Component 7 : 54
    Component 8 : 3
    Component 9 : 5

   
Distribution of **Spectral explanation**

    Component 0 : 0
    Component 1 : 0
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 92
    Component 6 : 6
    Component 7 : 1374
    
    
> How do we represent the above distribution? Table or bar-chart or what? First we need to be showing this on the training data to claim what the CNN has learned from the training data. Then we move on to prove the hypothesis on the development and the evaluation data.

    
On average, the explanation distribution show strong emphasis in the center of the 3 second audio signal (between 0.9 - 2.1secs). This corresponds to the region in audio signal where speech onset is detected. We find that majority of genuine audio files in the development data has non-speech frames in the beginning and speech onset in majority of cases starts somewhere around 900ms. On the spectral explanation distribution, it gives a very clear explanation that the model is looking at high frequency information above 7 kHz. 

> Therefore, what **conclusion** we get from above is that the model is identifying a file as genuine using information above 7 kHz mostly in the middle of the audio signal.
 






